In [1]:
import numpy as np
import itertools
import random
from statistics import stdev
import math
import ast

In [2]:
p_running = 0.9
p_failure = 1 - p_running
path = '/home/dkabe/Model_brainstorming/SAA_Analysis_v3/Instance_1/'

In [3]:
N = [32, 64, 96, 128, 160, 192, 256]
batches = 30
Products = 3
Market = 29
demand_realization = 25
text_file = open(path + 'Disruption_scenarios.txt', "r")
ls = text_file.read().split('\n')[:-1]
Scenarios = list(map(lambda x: ast.literal_eval(x), ls))
Probabilities = np.loadtxt(path + "disruption_probabilities.txt")
num_disruption = len(Scenarios)

In [4]:
p1_mean = 103500
p2_mean = 101500
p3_mean = 99750

p1_sd = 1650
p2_sd = 4125
p3_sd = 2000

distribution_mat = [[p1_mean, p1_sd],
                  [p2_mean, p2_sd],
                  [p3_mean, p3_sd]]

In [5]:
np.random.seed(1)
demand = np.zeros((demand_realization, Products, Market))
for r in range(demand_realization):
    demand[r][0] = np.round(np.random.normal(distribution_mat[0][0], distribution_mat[0][1], size = 29))
    demand[r][1] = np.round(np.random.normal(distribution_mat[1][0], distribution_mat[1][1], size = 29))
    demand[r][2] = np.round(np.random.normal(distribution_mat[2][0], distribution_mat[2][1], size = 29))
#np.savetxt("/home/dkabe/Model_brainstorming/SAA_Analysis_v3/Instance_1/demand.txt", demand.reshape(25*3, 29))

In [6]:
S = demand_realization*Scenarios
S_demand = []
S_prob = list(Probabilities)*demand_realization
multiplier = sum(S_prob)
S_prob = list(map(lambda x: x/multiplier, S_prob))
for r in range(demand_realization):
    S_demand+=([demand[r]]*num_disruption)

In [7]:
random.seed(1)
evaluation_indices = random.sample(range(len(S)), 3200)
evaluation_set = [S[index] for index in evaluation_indices]
evaluation_demand = np.array([S_demand[index] for index in evaluation_indices])
eval_probabilities = [S_prob[index] for index in evaluation_indices]
p_factor = sum(eval_probabilities)
eval_probabilities = list(map(lambda x: x/p_factor, eval_probabilities))
#with open(path + 'Evaluation_Set.txt', 'w') as f:
#    for item in evaluation_set:
#        f.write("%s\n" % item)
#f.close()
#np.savetxt(path + "p_scen.txt", eval_probabilities)
#np.savetxt("eval_set_demand.txt", evaluation_demand.reshape((3200*3,29)))

In [8]:
random.seed(1)
for num_scen in N:
    for batch in range(batches):
        indices = random.sample(range(len(S)), num_scen)
        sample_scenarios = [S[index] for index in indices]
        p_sample_scenarios = [S_prob[index] for index in indices]
        demand = np.array([S_demand[index] for index in indices])
        p_factor = 1/np.sum(p_sample_scenarios)
        p_sample_scenarios = list(map(lambda x: x*p_factor, p_sample_scenarios))  
        with open(path + 'Scenarios/' + str(num_scen) + '_' + str(batch) + '.txt', 'w') as f:
            for item in sample_scenarios:
                f.write("%s\n" % item)
        f.close()
        np.savetxt(path + 'Scen_probabilities/' + 'p_scen_' + str(num_scen) + '_' + str(batch) + '.txt', p_sample_scenarios)
        np.savetxt(path + 'Scen_demand/' + 'demand_' + str(num_scen) + '_' + str(batch) + '.txt', demand.reshape((num_scen*Products, Market)))

In [6]:
# version 2
path = '/home/dkabe/Model_brainstorming/SAA_Analysis_v3/Instance_1/'
N = [32, 64, 96, 128, 160, 192, 256]
p_scen = np.loadtxt('/home/dkabe/Model_brainstorming/SAA_Analysis_v3/Instance_1/p_scen.txt')[:1000]
p_factor = 1/np.sum(p_scen)
p_scen = list(map(lambda x: x*p_factor, p_scen))  

p_factor = 1/np.sum(p_scen)
p_scen = list(map(lambda x: x*p_factor, p_scen))  
for scen in N:
    gaps = []
    ub_averages = []
    ub_max = []
    objvals_lb = np.loadtxt(path + "Objectives/" + str(scen) + '_results.txt')
    avg_lb = (np.mean(objvals_lb))
    std_lb = (stdev(objvals_lb))
    lbwidth = std_lb/math.sqrt(batches)*2.045
    lbmin = round(avg_lb - lbwidth,2)
    #print(avg_lb, lbmin)
    #print('ci on lower bound = [', round(avg_lb-lbwidth,2), ',', round(avg_lb+lbwidth,2), ']')
    for batch in range(batches):
        objvals_ub = np.loadtxt(path + "Upper_Bounds/" + str(scen) + "_scenarios/" + str(scen) + "_" + str(batch) + "_UB_results.txt")[:1000]
        #objvals_ub = np.array([objvals_ub[index] for index in indices])
        avg_ub = (np.sum(p_scen*objvals_ub))
        std_ub = math.sqrt(np.sum(p_scen*((objvals_ub - avg_ub)**2)))
        ubwidth = std_ub/math.sqrt(1000)*1.96
        ubmax = round(avg_ub + ubwidth, 2)
        gap = round(100*((ubmax - lbmin)/lbmin),2)
        ub_averages.append(avg_ub)
        gaps.append(gap)
        ub_max.append(ubmax)
    min_gap = min(filter(lambda x: x >= 0, gaps))
    #min_gap = min(gaps)
    index = gaps.index(min_gap)
    #print(ub_averages[index], ub_max[index], min_gap)
    print(avg_lb, "\t", ub_averages[index], "\t", lbmin, "\t", ub_max[index], "\t", min_gap)

17560572.324333336 	 17643898.77267029 	 16423268.64 	 18069862.91 	 10.03
18246255.588 	 18049829.276163 	 17294782.48 	 18394645.46 	 6.36
17454241.925 	 17793197.155290376 	 16782849.76 	 18245364.87 	 8.71
18723160.650666665 	 17676809.69082504 	 17989219.5 	 18098836.03 	 0.61
18286059.542666662 	 17724846.743970968 	 17758870.8 	 18155955.02 	 2.24
18336696.168333333 	 17693901.600055724 	 17774297.99 	 18128826.08 	 1.99
18348320.432666667 	 17643898.77267029 	 17780372.91 	 18069862.91 	 1.63


In [21]:
p_scen = np.loadtxt('/home/dkabe/Model_brainstorming/SAA_Analysis_v3/Instance_1/p_scen.txt')

indices = random.sample(range(3200), 1000)
p_scen [p_scen[index] for index in indices]

p_scen = random.sample(list(p_scen), 1000)
p_factor = 1/np.sum(p_scen)
p_scen = list(map(lambda x: x*p_factor, p_scen))  

In [23]:
np.sum(p_scen)

0.9999999999999999